# Введение в машинное обучение.
# Построение модели анализа поведения пользователей мобильной связи

## Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».  

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи **классификации**, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.  

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- **сalls** — количество звонков,
- **minutes** — суммарная длительность звонков в минутах,
- **messages** — количество sms-сообщений,
- **mb_used** — израсходованный интернет-трафик в Мб,
- **is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Шаг 1. Откройте файл с данными и изучите его.

In [1]:
# Импорт нужных библиотек и классов
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
# Алгоритм классификации - решающее дерево
from sklearn.tree import DecisionTreeClassifier
# Алгоритм классификации - случайный лес
from sklearn.ensemble import RandomForestClassifier
# Алгоритм классификации - логистическая регрессия
from sklearn.linear_model import LogisticRegression

In [2]:
ub_df = pd.read_csv('users_behavior.csv')

In [3]:
ub_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
ub_df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


По датасету видно, что все столбцы с вещественным типом данных можно перевести в целочисленный (округлив дробные числа в большую сторону, т.к. обычно оператор всё равно это делает сам). Пропусков в данных нет.

In [5]:
# Округляем минуты и интернет-трафик
ub_df['minutes'] = ub_df['minutes'].apply(np.ceil)
ub_df['mb_used'] = ub_df['mb_used'].apply(np.ceil)

# Переводим типы в целочисленные
ub_df = ub_df.astype('int')

По данным видно, что признаками являются столбцы calls, minutes, messages и mb_used. А целевым признаком является столбец is_ultra, который принимает значения 0 либо 1.

## Шаг 2. Разделите исходные данные на обучающую, валидационную и тестовую выборки.

In [6]:
# Сначала выделим features и target признаки
features = ub_df.drop(['is_ultra'], axis=1)
target = ub_df['is_ultra']

Делить исходные данные будем в пропорциях: Обучающая выборка 60%, Валидационная 20% и Тестовая 20%  
Для этого воспользуемся методом train_test_split из библиотеки sklearn. Она делит исходные данные на 2 части, следовательно применять её надо 2 раза.  
Сначала поделим выделим тестовую выборку (20% от исх.), а потом из большего остатка (80% от исх.) выделим обучающую и валидационную выборки (по 75% и 25% от остатка соответственно).

In [7]:
features_comb, features_test, target_comb, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(features_comb, target_comb, test_size=0.25, random_state=12345)

## Шаг 3. Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

### Модель решающего дерева
Первая модель, которую рассмотрим будет модель решающего дерева. Будем менять в цикле гиперпараметр max_depth и искать лучший вариант среди получившихся моделей по accuracy. Также будем сохранять лучший вариант модели в файле 'best_dtree_model.joblib'

In [8]:
best_accuracy = 0
best_depth = 0
for i in range(1,21):
    model = DecisionTreeClassifier(max_depth=i, random_state=12345)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predicted_valid)
    if(accuracy > best_accuracy):
        best_accuracy = accuracy
        best_depth = i
        joblib.dump(model, 'best_dtree_model.joblib')
print('Best DecisionTreeClassifier model with max_depth =', best_depth, 'and accuracy =', best_accuracy)

Best DecisionTreeClassifier model with max_depth = 10 and accuracy = 0.7807153965785381


Таким образом видим, что самое большое значение accuracy 0.78 (доля правильных ответов) для валидационной выборки получили при максимальной глубине дерева равной 10.

### Модель случайного леса
Следующая модель - это модель случайного леса. Запустим 2 цикла (один вложенный в другой): по гиперпараметру max_depth (макс. глубина дерева,  четные с 2 до 30) и по n_estimators (количество деревьев в лесу, с 1 до 15). Также будем сохранять лучший вариант модели в файле 'best_rand_forest_model.joblib'

In [9]:
best_accuracy = 0
best_depth = 0
best_estim = 0
for i in range(2, 31, 2):
    for estim in range(1, 16):
        model = RandomForestClassifier(n_estimators=estim, max_depth=i, random_state=12345)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predicted_valid)
        if(accuracy > best_accuracy):
            best_accuracy = accuracy
            best_depth = i
            best_estim = estim
            joblib.dump(model, 'best_rand_forest_model.joblib')
print('Best RandomForestClassifier model with max_depth =', best_depth, ', best_estim =', best_estim, 'and accuracy =', best_accuracy)

Best RandomForestClassifier model with max_depth = 12 , best_estim = 10 and accuracy = 0.8009331259720062


Таким образом видим, что по модели случайного леса самое большое значение accuracy 0.8 для валидационной выборки получили при максимальной глубине дерева равной 12 и количестве деревьев 10. Значение лучше, чем у решающего дерева.

### Модель логистической регрессии
Наконец, построим модель по алгоритму логистической регрессии. Сохраним модель в файле 'lr_model.joblib'

In [10]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predicted_valid)
joblib.dump(model, 'lr_model.joblib')
print('LogisticRegression model with accuracy =', accuracy)

LogisticRegression model with accuracy = 0.6936236391912908


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


По модели логистической регрессии доля правильных ответов на валидационной выборке получилось худшим из 3 алгоритмов. Всего 0.69.

### Вывод по шагу 3.
Из 3-х рассмотренных типов моделей получили что самая лучшая доля правильных ответов на валидационной выборке у модели случайного леса с гиперпараметрами max_depth = 12 и n_estimators = 10.

## Шаг 4. Проверьте качество модели на тестовой выборке.
Проверим каждую из 3-х полученных моделей на тестовой выборке и найдем значение accuracy.

In [11]:
model = joblib.load('best_dtree_model.joblib')
predicted_test = model.predict(features_test)
accuracy = accuracy_score(target_test, predicted_test)
print('DecisionTreeClassifier model accuracy =', accuracy, '\n')

model = joblib.load('best_rand_forest_model.joblib')
predicted_test = model.predict(features_test)
accuracy = accuracy_score(target_test, predicted_test)
print('RandomForestClassifier model accuracy =', accuracy, '\n')

model = joblib.load('lr_model.joblib')
predicted_test = model.predict(features_test)
accuracy = accuracy_score(target_test, predicted_test)
print('LogisticRegression model accuracy =', accuracy, '\n')

DecisionTreeClassifier model accuracy = 0.7667185069984448 

RandomForestClassifier model accuracy = 0.7884914463452566 

LogisticRegression model accuracy = 0.6967340590979783 



### Вывод по шагу 4.
Видим, что и на тестовой выборке лучшей оказалась модель случайного леса. Значение accuracy у нее 0.788.
Этот вывод подтверждает сравнительную таблицу моделей в тренажере. Там было сказано, что случайный лес обладает самым высоким качеством при низкой скорости работы модели.

## Шаг 5. Дополнительное задание: проверьте модели на вменяемость.

Для проверки на вменяемость сравним модель со случайной моделью, которая выдает равновероятно значения для столбца is_ultra 0 или 1 (с вероятностью 50%).  
Для начала посмотрим в каком соотношении представлены значения 1 и 0 в общем наборе целевого признака тестовой выборки target_test.

In [12]:
target_test.value_counts(normalize=True)

0    0.695179
1    0.304821
Name: is_ultra, dtype: float64

Можно сказать, что 70% это значения 0 и 30% значения 1.  
Получаем формулу для accuracy случайной модели:  

accuracy = 0.7x0.5 + 0.3x0.5 = 0.5

Видим, что accuracy всех полученных нами моделей (0.697, 0.767, 0.788) выше чем accuracy случайной модели. Значит, полученные нами модели вполне вменяемы :-)